In [14]:
import torch
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
model_ft = models.densenet161(pretrained=True)
x = torch.randn([1,3,640 , 480])

model_ft.features._modules.items()
features = [x]
i=-2;
for k, v in (model_ft.features._modules.items()): 
  print(i)
  i+=1
  features.append( v(features[-1]) )
  # print(k)
  # print(v)
  # print(features[-1].shape)

for i,x in enumerate(features):
    print(i,x.shape)

-2
-1
0
1
2
3
4
5
6
7
8
9
0 torch.Size([1, 3, 640, 480])
1 torch.Size([1, 96, 320, 240])
2 torch.Size([1, 96, 320, 240])
3 torch.Size([1, 96, 320, 240])
4 torch.Size([1, 96, 160, 120])
5 torch.Size([1, 384, 160, 120])
6 torch.Size([1, 192, 80, 60])
7 torch.Size([1, 768, 80, 60])
8 torch.Size([1, 384, 40, 30])
9 torch.Size([1, 2112, 40, 30])
10 torch.Size([1, 1056, 20, 15])
11 torch.Size([1, 2208, 20, 15])
12 torch.Size([1, 2208, 20, 15])


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UpSample(nn.Sequential):
    def __init__(self, skip_input, output_features):
        super(UpSample, self).__init__()        
        self.convA = nn.Conv2d(skip_input, output_features, kernel_size=3, stride=1, padding=1)
        self.leakyreluA = nn.LeakyReLU(0.2)
        self.convB = nn.Conv2d(output_features, output_features, kernel_size=3, stride=1, padding=1)
        self.leakyreluB = nn.LeakyReLU(0.2)

    def forward(self, x, concat_with):
        up_x = F.interpolate(x, size=[concat_with.size(2), concat_with.size(3)], mode='bilinear', align_corners=True)
        return self.leakyreluB( self.convB( self.leakyreluA(self.convA( torch.cat([up_x, concat_with], dim=1) ) ) )  )

class Decoder(nn.Module):
    def __init__(self, num_features=2208, decoder_width = 0.5):
        super(Decoder, self).__init__()
        features = int(num_features * decoder_width)

        self.conv2 = nn.Conv2d(num_features, features, kernel_size=1, stride=1, padding=1)

        self.up1 = UpSample(skip_input=features//1 + 384, output_features=features//2)
        self.up2 = UpSample(skip_input=features//2 + 192, output_features=features//4)
        self.up3 = UpSample(skip_input=features//4 +  96, output_features=features//8)
        self.up4 = UpSample(skip_input=features//8 +  96, output_features=features//16)

        self.conv3 = nn.Conv2d(features//16, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, features):
        x_block0, x_block1, x_block2, x_block3, x_block4 = features[3], features[4], features[6], features[8], features[11]
        x_d0 = self.conv2(x_block4)
        x_d1 = self.up1(x_d0, x_block3)
        x_d2 = self.up2(x_d1, x_block2)
        x_d3 = self.up3(x_d2, x_block1)
        x_d4 = self.up4(x_d3, x_block0)
        return self.conv3(x_d4)

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()       
        import torchvision.models as models
        self.original_model = models.densenet161( pretrained=True )

    def forward(self, x):
        features = [x]
        for k, v in self.original_model.features._modules.items(): features.append( v(features[-1]) )
        return features

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        return self.decoder( self.encoder(x) )